## Jupyter Notebook for Coursera Capstone Project - Week 3 Assignment - Patricio Oria

### First, we import libraries 

In [42]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Libraries imported.
Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


### Defining my Foursquare credential and version

In [43]:
CLIENT_ID = '4OVOCEJWJHFPVYDP1OPZO4PQDDFQCRJQLID3KVHZR3PAHBBU' # your Foursquare ID
CLIENT_SECRET = 'Y4HHOUOEIXSX1RW2M2UUGV0F1FUZKR1CQLQGTOI51VMV5KKA' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4OVOCEJWJHFPVYDP1OPZO4PQDDFQCRJQLID3KVHZR3PAHBBU
CLIENT_SECRET:Y4HHOUOEIXSX1RW2M2UUGV0F1FUZKR1CQLQGTOI51VMV5KKA


### Web Scrapping the List of Postal Codes of Canada

In [44]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df=pd.read_html(url, header=0)[0]

df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)

df.head(11)


,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [45]:
df.shape

(103, 3)

### Grouping neighbourhoods in the same borough

In [55]:
df_grouped = df.groupby(['Postal Code', 'Borough'], as_index=False).agg(lambda x: ",".join(x))

df_grouped

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


### Defing the shape of the grouped dataframe

In [56]:
df_grouped.shape

(103, 3)

### Downloading the Geospatial Coordinates CSV and turning it into a dataframe

In [41]:
!wget -q -O "Geospatial_Coordinates.csv" http://cocl.us/Geospatial_data
print('Data downloaded!')

geo_df = pd.read_csv("Geospatial_Coordinates.csv")
geo_df

Data downloaded!


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


### Merging grouped dataframe and geospatial coordinates dataframe

In [62]:
df_merge = pd.merge(left=df_grouped, right=geo_df, left_on = 'Postal Code', right_on = 'Postal Code')

df_merge.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
